In [3]:
import torch.optim as optim
import torch.nn.functional as F

import bayescache.api as api
from bayescache.data import P3B3
from bayescache.models import mtcnn
from bayescache.api.source import TrainingData
from bayescache.callbacks.time_tracker import TimeTracker

In [4]:
hparams = mtcnn.Hyperparameters()
# Update hyperparameters for the Synthetic data.
hparams.vocab_size = 4014
hparams.max_sent_len = 1500

model = mtcnn.new(hparams)

optimizer = optim.RMSprop(model.parameters(), lr=7.0e-4, eps=1e-3)

In [5]:
learner = api.Learner(device='cpu', model=model)
#learner.summary()

In [8]:
train = P3B3(root='/Users/yngtodd/data', partition='train', download=True)
test = P3B3(root='/Users/yngtodd/data', partition='test', download=True)

source = TrainingData(train_source=train, val_source=test, num_workers=2, batch_size=1)

In [9]:
metrics = learner.metrics()
callbacks = [TimeTracker()]

training_info = api.TrainingInfo(
    start_epoch_idx=0,
    run_name='test',
    metrics=metrics,
    callbacks=callbacks
)

In [10]:
training_info.on_train_begin()

In [ ]:
for global_epoch_idx in range(training_info.start_epoch_idx + 1, 1 + 1):
    epoch_info = api.EpochInfo(
        training_info=training_info,
        global_epoch_idx=global_epoch_idx,
        batches_per_epoch=source.train_iterations_per_epoch(),
        optimizer=optimizer
    )

    # Execute learning
    learner.run_epoch(epoch_info, source)

### Diagnosing Loss

In [11]:
import torch
from torch.utils.data import DataLoader

In [12]:
trainloader = DataLoader(train)

In [13]:
for idx, (data, label) in enumerate(trainloader):
    logits = model(data)
    #print(len(logits))
    #print(logits)
    print(len(label))
    print(label.shape)
    #F.cross_entropy(logits[0], label[0])
    if idx == 0:
        break 

1
torch.Size([1, 4])


In [14]:
logits

[tensor([[-2.9519, -1.2010, -3.6190,  2.0904, -0.3148,  1.6946]],
        grad_fn=<AddmmBackward>),
 tensor([[ 2.2192, -1.9326]], grad_fn=<AddmmBackward>),
 tensor([[-0.7771,  1.7999]], grad_fn=<AddmmBackward>),
 tensor([[ 1.5455, -0.2997, -2.2939]], grad_fn=<AddmmBackward>)]

In [33]:
torch.tensor((label[0][0],))
label[:,2]

tensor([2])

In [34]:
F.cross_entropy(F.softmax(logits[1]), torch.tensor((label[:,1],)))

/Users/yngtodd/src/checkout/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor(0.3217, grad_fn=<NllLossBackward>)

In [ ]:
logits

In [ ]:
logits[0].shape
x = label[0][0].item()
target = torch.tensor((x,))
target

In [ ]:
input = torch.randn(1, 6, requires_grad=True)
target = torch.randint(6, (1,), dtype=torch.int64)

In [ ]:
input.shape
target

### Shape of Things to Come:

In [ ]:
data, labels = train.load_data()

In [ ]:
len(labels)

In [ ]:
len(labels[:,0])

In [ ]:
import numpy as np
unq, counts = np.unique(labels[:,0], return_counts=True)

In [ ]:
unq

In [ ]:
counts

In [ ]:
data[0].shape

In [ ]:
sum(counts)

In [ ]:
data.shape

In [ ]:
labels

In [ ]:
data